In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
import cv2
import numpy as np
import os

# 사전 학습된 Mask R-CNN 모델 로드
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()  # 모델을 평가 모드로 설정

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
# Haar Cascade 분류기 파일 경로
face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Haar Cascade 분류기 로드
face_cascade = cv2.CascadeClassifier(face_cascade_path)

for i in range(0,12):
    # 기존 이미지 로드
    original_image_path = f'/content/drive/MyDrive/3조/weddingdataset/wedding_test/image/{60000 + i}_00.jpg'

    # 이미지 파일이 존재하는지 확인
    if not os.path.exists(original_image_path):
        print(f"File {original_image_path} does not exist.")
        continue

    original_image = cv2.imread(original_image_path)
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # 회색 이미지 추출
    image_tensor = torchvision.transforms.functional.to_tensor(original_image_rgb).unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
        prediction = model(image_tensor)[0]

    # 결과 확인 (마스크와 클래스 ID 추출)
    masks = prediction['masks'].cpu().numpy()
    labels = prediction['labels'].cpu().numpy()
    scores = prediction['scores'].cpu().numpy()

    # 클래스 ID가 1인 마스크 필터링 (예: 사람)
    person_mask = np.zeros((original_image_rgb.shape[0], original_image_rgb.shape[1]), dtype=np.uint8)
    for j in range(masks.shape[0]):
        if labels[j] == 1 and scores[j] > 0.5:  # 신뢰도가 0.5 이상인 경우만 사용
            person_mask = np.maximum(person_mask, masks[j, 0])

    # 마스크 확장
    kernel = np.ones((30, 30), np.uint8)  # 커널 크기를 조정하여 확장 범위 조절
    dilated_mask = cv2.dilate(person_mask, kernel, iterations=1)

    # 하얀색과 검정색 정의
    white_color = (255, 255, 255)  # 하얀색
    black_color = (0, 0, 0)  # 검정색

    # 모든 영역을 검정색으로 채우기
    processed_image = np.full_like(original_image_rgb, black_color, dtype=np.uint8)

    # 확장된 영역을 하얀색으로 채우기
    processed_image[dilated_mask > 0.5] = white_color

    # 얼굴 감지
    gray = cv2.cvtColor(original_image_rgb, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        # 얼굴 위치가 가장 높은 것을 찾기
        highest_face = min(faces, key=lambda rect: rect[1])
        x, y, w, h = highest_face

        # 얼굴 중심과 반경 설정
        center_x, center_y = x + w // 2, y + h // 2

        # 너비와 높이에 각각 다른 배율 적용
        adjusted_w = int(1.8 * w)
        adjusted_h = int(1.2 * h)

        # 타원형 마스크 생성
        mask = np.zeros_like(original_image_rgb)
        mask = cv2.ellipse(mask, (center_x, center_y), (adjusted_w // 2, adjusted_h // 2), 0, 0, 360, (255, 255, 255), -1)

        # 얼굴 마스크 영역을 검정으로 채우기
        processed_image[mask[:, :, 0] == 255] = black_color

        # 중앙을 기준으로 윗부분을 검정으로 채우기
        processed_image[:center_y, :] = black_color

    # 결과 이미지 저장 (RGB 색상 공간으로 변환하여 저장)
    output_path = f'/content/drive/MyDrive/3조/weddingdataset/agnostic-mask/{60000 + i}_00_mask.png'
    cv2.imwrite(output_path, cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR))

In [ ]:
# Haar Cascade 분류기 파일 경로
face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Haar Cascade 분류기 로드
face_cascade = cv2.CascadeClassifier(face_cascade_path)

# 기존 이미지 폴더 경로
input_folder = '/content/drive/MyDrive/3조/weddingdataset/viton-hd-idm/train/image'
output_folder = '/content/drive/MyDrive/3조/weddingdataset/viton-hd-idm/train/agnostic-mask'

# 디렉토리 내 모든 파일 이름을 가져오기
file_names = os.listdir(input_folder)

for file_name in file_names:
    # 기존 이미지 경로
    original_image_path = os.path.join(input_folder, file_name)

    # 이미지 파일이 존재하는지 확인
    if not os.path.exists(original_image_path):
        print(f"File {original_image_path} does not exist.")
        continue

    # 이미지 로드
    original_image = cv2.imread(original_image_path)
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # 회색 이미지 추출
    image_tensor = torchvision.transforms.functional.to_tensor(original_image_rgb).unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
        prediction = model(image_tensor)[0]

    # 결과 확인 (마스크와 클래스 ID 추출)
    masks = prediction['masks'].cpu().numpy()
    labels = prediction['labels'].cpu().numpy()
    scores = prediction['scores'].cpu().numpy()

    # 클래스 ID가 1인 마스크 필터링 (예: 사람)
    person_mask = np.zeros((original_image_rgb.shape[0], original_image_rgb.shape[1]), dtype=np.uint8)
    for j in range(masks.shape[0]):
        if labels[j] == 1 and scores[j] > 0.5:  # 신뢰도가 0.5 이상인 경우만 사용
            person_mask = np.maximum(person_mask, masks[j, 0])

    # 마스크 확장
    kernel = np.ones((30, 30), np.uint8)  # 커널 크기를 조정하여 확장 범위 조절
    dilated_mask = cv2.dilate(person_mask, kernel, iterations=1)

    # 하얀색과 검정색 정의
    white_color = (255, 255, 255)  # 하얀색
    black_color = (0, 0, 0)  # 검정색

    # 모든 영역을 검정색으로 채우기
    processed_image = np.full_like(original_image_rgb, black_color, dtype=np.uint8)

    # 확장된 영역을 하얀색으로 채우기
    processed_image[dilated_mask > 0.5] = white_color

    # 얼굴 감지
    gray = cv2.cvtColor(original_image_rgb, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        # 얼굴 위치가 가장 높은 것을 찾기
        highest_face = min(faces, key=lambda rect: rect[1])
        x, y, w, h = highest_face

        # 얼굴 중심과 반경 설정
        center_x, center_y = x + w // 2, y + h // 2

        # 너비와 높이에 각각 다른 배율 적용
        adjusted_w = int(1.8 * w)
        adjusted_h = int(1.2 * h)

        # 타원형 마스크 생성
        mask = np.zeros_like(original_image_rgb)
        mask = cv2.ellipse(mask, (center_x, center_y), (adjusted_w // 2, adjusted_h // 2), 0, 0, 360, (255, 255, 255), -1)

        # 얼굴 마스크 영역을 검정으로 채우기
        processed_image[mask[:, :, 0] == 255] = black_color

        # 중앙을 기준으로 윗부분을 검정으로 채우기
        processed_image[:center_y, :] = black_color

    # 파일 이름 변경 (뒤에 _mask 추가하고 확장자 png로 변경)
    base_name, _ = os.path.splitext(file_name)
    output_file_name = f"{base_name}_mask.png"
    output_path = os.path.join(output_folder, output_file_name)

    # 결과 이미지 저장 (RGB 색상 공간으로 변환하여 저장)
    cv2.imwrite(output_path, cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR))